In [1]:
import cdsapi
import xarray as xr
import datetime
from ipyfilechooser import FileChooser
import rioxarray as rxr
import xarray as xr
import fierpy
import glob
import pandas as pd
from pathlib import Path
from ipyfilechooser import FileChooser
import re
from fier_local import reof as freof
from fier_local import sel_best_fit

import opensarlab_lib as asfn
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates

In [2]:
def get_dates(dir_path, prefix):
    dates = []
    pths = list(dir_path.glob(f'{prefix}.tif*'))

    for p in pths:
        date_regex = '\d{8}'
        date = re.search(date_regex, str(p))
        if date:
            dates.append(date.group(0))
    return dates

In [3]:
fc = FileChooser(Path.cwd())
display(fc)

FileChooser(path='/home/jovyan/Hydrosar', filename='', title='', show_hidden=False, select_desc='Select', chan…

In [5]:

tiff_dir = Path(fc.selected)/'RTC_GAMMA'
tiffs = list(tiff_dir.glob(f'*VV.tif*'))

times = get_dates(tiff_dir, '*VV')
times.sort()
times = pd.DatetimeIndex(times)
times.name = "time"

da = xr.concat([rxr.open_rasterio(f) for f in tiffs], dim=times)

info = gdal.Info(str(tiffs[0]), format='json')
info = info['coordinateSystem']['wkt']
utm = info.split('ID')[-1].split(',')[1][0:-2]

da.attrs['epsg'] = utm



from pyproj import Transformer
points = [(da.y.min().values, da.x.min().values), (da.y.max().values, da.x.max().values)]  
boundaries = []
transformer = Transformer.from_crs(int(da.epsg), 4326)
for pt in transformer.itransform(points): boundaries.append(pt)
boundaries = np.array(boundaries)

/home/jovyan/.local/envs/hydrosar/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning:

Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.



In [9]:

# Create a folder to host the ERA5 data
weather_data_folder = Path(fc.selected)/'ERA5'
weather_data_folder.mkdir(exist_ok=True)


# Create a new instance of the CDS API client
# Define the API URL and key
api_url = 'https://cds.climate.copernicus.eu/api/v2'
api_key = '105035:a341bca0-1659-4056-a656-f4d505fffd4c'


# Create the API client
c = cdsapi.Client(url=api_url, key=api_key)

result = []
min_date = times.min()
max_date = times.max()


for i in range(times.min().year, times.max().year+1):
    c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': 'total_precipitation',
                'format': 'netcdf',
                'year': i,
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
                'area': [boundaries[0,0], boundaries[0,1], boundaries[1,0], boundaries[1,1]],  # Global
            },
            f'{weather_data_folder}/{i}.nc',
    )

    # Open precipitation dataset
    filename = f'{weather_data_folder}/{i}.nc'
    # Resample for 1 value per day, in-between are summed
    ds = xr.open_dataset(filename).resample(time='D').sum()
    # Create a time mask, based on the validity of layers and the custom date-range
    t_mask = np.logical_and(ds.time.values>da.time.min().values, ds.time.values<da.time.max().values)
    # Select only the values in the times array
    ds = ds.sel(time=t_mask)    
    # Append for concatenation later
    result.append(ds)

combined = xr.concat(result, dim='time')


# Create the new array with the timestamps from `da`
arr = np.zeros((len(da.time)))

# Iterate over the timestamps in `da`
for i in range(len(da.time)-1):
    # Find the values in `ds` that fall between the current timestamp and the next timestamp
    values_between = combined.sel(time=slice(da.time[i], da.time[i+1]))

    # Sum the values between the timestamps
    summed_values = np.nansum(values_between.tp.sum(dim='time').values)

    # Assign the summed value to the corresponding timestamp in the new dataset
    arr[i] = summed_values


# Save array as netcdf
era_ds = xr.DataArray(arr, coords={"time": da.time}, dims="time").to_netcdf(f'{weather_data_folder}/era5_data.nc')


2023-06-21 23:09:33,663 INFO Welcome to the CDS
INFO:cdsapi:Welcome to the CDS
2023-06-21 23:09:33,664 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
INFO:cdsapi:Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-06-21 23:09:33,928 INFO Request is queued
INFO:cdsapi:Request is queued


HTTPError: 404 Client Error:  for url: https://cds.climate.copernicus.eu/api/v2/tasks/f1b14b02-da6b-4fad-8370-4f2ebb97e7cf